In [1]:
import requests
import os
import json
import pandas as pd
from datetime import datetime, timedelta
import csv
import time
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
from nltk.tokenize import TweetTokenizer
from nltk.lm.preprocessing import pad_both_ends, padded_everygram_pipeline
import re
from nltk.lm import KneserNeyInterpolated, Vocabulary
import demoji 
from nltk.lm.smoothing import KneserNey
import functools
from nltk.util import everygrams
from nltk.util import bigrams, ngrams
from collections import Counter 
import copy
from numba import njit

In [113]:
"""# src: https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a
def auth():
    return os.getenv('TOKEN')

def create_headers(bearer_token):
    headers = {"Authorization": f"Bearer {bearer_token}"}
    return headers


def create_url(query, max_results= 100):
    url = "https://api.twitter.com/2/tweets/search/recent"
    query_params = {"query": f"{query} lang:en", "expansions": "geo.place_id",
                    "max_results": max_results, 
                    "place.fields": "country_code,full_name,geo",
                    "tweet.fields": "author_id",
                   "next_token": {}}
    return (url, query_params)
    

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code) + "\n")
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

"""

In [118]:
"""keyword = "football"
max_results = 100
bearer_token = auth()
headers = create_headers(bearer_token)
url, query_params = create_url(keyword, max_results)"""

In [119]:
"""csvFile = open("football.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['author id', 'tweet id', 'text', 'geo', 'country_code', 'full_name', 'bbox'])"""

57

In [120]:
"""count = 0
json_response = connect_to_endpoint(url, headers, query_params)
amnt = 0

while(count < 1000):
    count += amnt
    for step, tweet in enumerate(json_response['data']):
        author_id = tweet['author_id']
        tweet_id = tweet['id'] 
        text = tweet['text'] 
        if ('geo' in tweet):   
            geo = tweet['geo']['place_id']
        else:
            geo = " "
        try: 
            country_code = json_response['includes']['places'][step]['country_code']
        except:
            country_code = " "
        try: 
            full_name = json_response['includes']['places'][step]['full_name']
        except:
            full_name = " "
        try: 
            bbox = json_response['includes']['places'][step]['geo']
        except:
            bbox = " "

        res = [author_id, tweet_id, text, geo, country_code, full_name, bbox]
        csvWriter.writerow(res)
    
    amnt = json_response['meta']['result_count']
        
    json_response = connect_to_endpoint(url, headers, query_params, 
                                        next_token = json_response['meta']['next_token'])"""

Endpoint Response Code: 200

Endpoint Response Code: 200

Endpoint Response Code: 200

Endpoint Response Code: 200

Endpoint Response Code: 200

Endpoint Response Code: 200

Endpoint Response Code: 200

Endpoint Response Code: 200

Endpoint Response Code: 200

Endpoint Response Code: 200

Endpoint Response Code: 200

Endpoint Response Code: 200



In [404]:
#https://stackoverflow.com/questions/25363498/is-there-a-standard-approach-to-returning-values-from-coroutine-endpoints
class FINISH_PROCESSING_SIGNAL(Exception): pass
tknzr = TweetTokenizer(strip_handles=False, reduce_len=True)

def tokenize_remove_emoticons(elem):
    tokenized_list = tknzr.tokenize(elem)
    # replacing emojis with their description, 
    # after tokenizer so as not to make them seperate words
    return list(map(lambda x: demoji.replace_with_desc(x), tokenized_list))
def apply_over(arr):
    return np.array([[tokenize_remove_emoticons(x) for x in lst] for lst in arr], dtype = object)

# NLTK is a crap library, and so I had to do this manually 
def flatten_2d_lst(lst):
    train_flattened = []
    for x in lst: 
        for j in x: 
            train_flattened.append(j)
    return train_flattened

def coroutine(func):
    @functools.wraps(func)
    def start(*args, **kwargs):
        cr = func(*args, **kwargs)
        next(cr)
        return cr
    return start

def source(df, targets):
    for target in targets: 
        target.send( df)
        processed_data = target.throw(FINISH_PROCESSING_SIGNAL)
        return processed_data
    
"""
Note, these transformations are completely deterministic
irrespective of the dataset being used, thus we can apply 
this initial part on the entire datafram."""
@coroutine  
def clean_train_test_split(targets):
    try:
        while True: 
            df = (yield)
            # removing all ethereum addresses
            df.text = df.text.apply(lambda x: re.sub(r'0x[a-fA-F0-9]{40}', 'ethe_addy', x)) 
            
            # replace all dollar amounts with vague "dollar_amnt" 
            df.text = df.text.apply(lambda x: re.sub(r'\$[0-9]+(.[0-9]+)?', 'dollar_amnt', x)) 
            df.text = df.text.apply(lambda x: re.sub(r'\d', 'random_num', x)) 
            df.text = df.text.apply(lambda x: re.sub(r'(random_num)*random_num', 'random_num', x))
            
            #lowercasing all letters
            df['text'] = df.text.str.lower()
            # will keep the handles for the sake of n-gram approximation, but specifics don't matter
            df.text = df.text.apply(lambda x: re.sub(r"@([a-zA-Z0-9_]{1,50})", 'randomhandle', x))
            # removing url links
            df.text = df.text.apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
            df.text = df.text.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
            # removing embedded links, videos
            df.text = df.text.apply(lambda x: re.sub(r'{link}', '', x))
            df.text = df.text.apply(lambda x: re.sub(r"\[video\]", '', x))
            train, test = train_test_split(df, test_size=1000, random_state=42)
            for target in targets: 
                target.send( (train, test) )
    except FINISH_PROCESSING_SIGNAL as err:
        yield target.throw(err)
        
@coroutine  
def train_tokenizer(targets):
    try:
        while True: 
            trainer = PunktTrainer()
            train, test = (yield)
            array_strs =train[['text']].values.flatten()
            array_strs_test =test[['text']].values.flatten()
            map(lambda x: trainer.train(x), array_strs)
            trainer.finalize_training()
            tokenizer = PunktSentenceTokenizer(trainer.get_params())
            for target in targets: 
                target.send( (tokenizer, array_strs, array_strs_test) )
    except FINISH_PROCESSING_SIGNAL as err:
        yield target.throw(err)
            
@coroutine  
def segment_sentences(targets):
    global temp1, temp2
    try:
        while True: 
            tokenizer, x, test = (yield)
            segment_sentences = np.array([tokenizer.tokenize(xi) for xi in x], dtype = object)
            segment_sentences_test = np.array([tokenizer.tokenize(xi) for xi in test], dtype = object)
            temp1, temp2 = segment_sentences, segment_sentences_test
            for target in targets: 
                target.send( (segment_sentences, segment_sentences_test))
    except FINISH_PROCESSING_SIGNAL as err:
        yield target.throw(err)

@coroutine   
def sentence_tokenizer(targets):
    try:
        while True: 
            x, test = (yield)
            x, test = apply_over(x), apply_over(test)
            flattened, flattened_test = [], []
            for xi in x:
                flattened.extend(xi)
            for xi in test:
                flattened_test.extend(xi)
            for target in targets: 
                target.send( (flattened, flattened_test) )
    except FINISH_PROCESSING_SIGNAL as err:
        yield target.throw(err)
        
@coroutine   
def unk_placement(targets):
    try:
        while True: 
            x, test = (yield)
            counter = Counter(flatten_2d_lst(x))
            filtered_list = Counter({k: c for k, c in counter.items() if c >= 2})
            filtered_set = set(filtered_list.keys())
            for step, xi in enumerate(x): 
                for step2, j in enumerate(xi): 
                    if j not in filtered_set: 
                        x[step][step2] = '<UNK>'
            for target in targets:
                target.send( (x, test) )
    except FINISH_PROCESSING_SIGNAL as err:
        yield target.throw(err)
        
@coroutine
def sink():
    data = {}
    try:
        while True:
            data = (yield)
    except FINISH_PROCESSING_SIGNAL:
        yield data

def start_generator(csv_name):
    a = sink()
    b = unk_placement(targets = [a])
    c = sentence_tokenizer(targets = [b])
    d = segment_sentences(targets = [c])
    e = train_tokenizer(targets = [d])
    f = clean_train_test_split(targets = [e])
    df = pd.read_csv(csv_name)
    train, test = source(df, targets = [f])
    return (train, test)

In [399]:
def gen_new_padding(n, dataset): 
    train_copy = copy.deepcopy(dataset)
    for step, x in enumerate(train_copy): 
        train_copy[step] = list(pad_both_ends(x, n=n))
        train_copy[step] = list(ngrams(train_copy[step], n=n))
    return train_copy

def gen_new_model(n):
    global train_vocab
    train_copy = gen_new_padding(n,train)
    test_copy = gen_new_padding(n,test)
    train_vocab = Counter(flatten_2d_lst(train_copy))
    model = KneserNeyInterpolated(n, discount=0.1)
    model.fit(train_copy , train_vocab)
    return train_copy, test_copy, model 

def get_log_frequency(**kwargs):
    context_one, word = kwargs['context_one'], kwargs['word']
    if word in context_one: 
        count = context_one[word]
    else:
        count = context_one['<UNK>',]
    return np.log2(count/sum(context_one.values()))

def get_log_frequency_bigram(**kwargs):
    context_one, word, context_two = kwargs['context_one'], kwargs['word'], kwargs['context_two']
    if word in context_two: 
        count = context_two[word]
        if (word[0],) not in context_one:  # <s>
            total_count = padded_amount
        else: 
            total_count = context_one[(word[0],)]
        return np.log2(count/ total_count)
    # StupidBackoff 
    return np.log2(.8) + get_log_frequency( word = (word[1],), context_one = context_one ) 

def get_log_frequency_trigrams(**kwargs):
    context_one, word, context_two = kwargs['context_one'], kwargs['word'], kwargs['context_two']
    context_three = kwargs['context_three']
    if word in context_three: 
        count = context_three[word]
        previous_two_words = word[:2]
        if previous_two_words not in context_two:  # <s>, <s>
            total_count = padded_amount
        else: 
            total_count = context_two[previous_two_words]
        return np.log2(count/ total_count)
    return np.log2(.8) + get_log_frequency_bigram(word = word[1:], \
                                                      context_one = context_one, context_two = context_two)


def print_perplexity_scores_helper(n, log_scorer, dataset, n_gram_type, context):    
    perplexity = lambda j: np.power(2, - np.mean([log_scorer(
        context_one = context[1], context_two = context[2], context_three = context[3],\
        word = i) for i in j]))
    perplexities_avg = sum(list(map(lambda x: perplexity(x), dataset)))/len(dataset)
    print(f"The average {n_gram_type} perplexity score of any test set tweet is : {perplexities_avg} ")
    
    
def print_perplexity_scores():
    global padded_amount
    context = {1: {}, 2: {}, 3: {}}
    funcs = {1: get_log_frequency, 2: get_log_frequency_bigram, 3: get_log_frequency_trigrams}
    decs = {1: "unigram", 2: "bigram", 3: "trigram"}
    for n in range(1, 4):             
        train_set, test_set, model = gen_new_model(n)
        context[n] = copy.deepcopy(model.vocab.counts)
        if n == 2: 
            padded_amount = sum(Counter({k: c for k, c in model.vocab.counts.items() \
                                   if k[1] == '</s>'}).values()) # symmetrical with <s>
        print_perplexity_scores_helper(n, funcs[n], test_set, decs[n], context)

In [400]:
# NFTs
train, test = start_generator("nfts.csv")
print_perplexity_scores()

The average unigram perplexity score of any test set tweet is : 670.2454773666304 
The average bigram perplexity score of any test set tweet is : 58.82761504218191 
The average trigram perplexity score of any test set tweet is : 35.109474179202245 


In [310]:
# Football
_, test = start_generator("football.csv")
print_perplexity_scores()

The average unigram perplexity score of any test set tweet is : 916.0631700646361 
The average bigram perplexity score of any test set tweet is : 282.4735987661044 
The average trigram perplexity score of any test set tweet is : 233.4433476582671 


In [355]:
_,_, model = gen_new_model(1)

In [410]:
f = open('codes.json')
emoji_codes = json.load(f)
f.close()

emoji_codes = {k: c for c,k in emoji_codes.items() }
emoji_codes

In [ ]:
# unigram, just pick a random word 

# bigram, pick a random first word with <s> , then use the end of that word to generate the next
# and so on until hitting a </s>

#trigram, same idea, except a jump of 3 


In [ ]:
import random 

def unigram_generator(length = 10):
    string = ''
    for i in range(10):
        random_choice = random.choices(*zip(*train_vocab.items()))[0][0]
        if rand_choice in emoji_codes:
            rand_choice = emoji_codes[rand_choice]
        elif random_choice == 'ethe_addy':
            rand_choice = something
        elif random_choice == 'dollar_amnt':
            rand_choice = something
        elif rand_choice = 'random_num'
            rand_choice = something 
        elif rand_choice = 'randomhandle'
            rand_choice = something
            
        string += rand_choice
    
    